In [9]:
import numpy as np

from bokeh.io import push_notebook
from bokeh.layouts import gridplot
from bokeh.layouts import row, widgetbox
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Range1d,CustomJS,Slider
from scipy.interpolate import interp1d
from bokeh.models.widgets import Slider
from ipywidgets import interact

def entropy(T,a,L,k,Tc):
    return a+536*(np.log(T)-np.log(270))+L/(1+np.exp(k*(T-Tc)))

T=np.linspace(270,330,50)

Tshift  = 6
Tc0     = 295 
Thyst   = .75
k0T     = -.5
k1T     = -.5
L       = 40
a       =0.5
Moff    = -2

################### Entropy Plot
p2 = figure(title="Specific Entropy")
p2.grid.grid_line_alpha = 0.3
p2.xaxis.axis_label = 'Temperature (K)'
p2.yaxis.axis_label = 'Specific Entropy (J/kgK)'
p2.x_range = Range1d(275, 320)
p2.y_range = Range1d(20, 150)

sh0 = entropy(T,a,L,k,Tc0+Thyst)
sc0 = entropy(T,a,L,k,Tc0-Thyst) 
sh1 = entropy(T,a+Moff,L,k,Tc0+Thyst+Tshift) 
sc1 = entropy(T,a+Moff,L,k,Tc0-Thyst+Tshift) 

r1=p2.line(T, sh0, legend='s_{h,lf}', color="green")
r2=p2.line(T, sc0, legend='s_{c,lf}', color="blue")
r3=p2.line(T, sh1, legend='s_{h,hf}', color="red")
r4=p2.line(T, sc1, legend='s_{c,hf}', color="purple")

p2.legend.location = "top_left"



################### Specific Heat PLot

# make diff fun
def differ(x,y):
    dy = np.zeros(y.size-1,np.float)
    dy = np.diff(y)/np.diff(x)
    return dy
# first differentate the entropye curve
Tcp=T[0:-1]
ch0= differ(T,sh0)*Tcp
cc0= differ(T,sc0)*Tcp
ch1= differ(T,sh1)*Tcp
cc1= differ(T,sc1)*Tcp

p1 = figure(title="Specific Heat")
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'Temperature (K)'
p1.yaxis.axis_label = 'Specific Heat (J/kgK)'
p1.x_range = Range1d(275, 310)
p1.y_range = Range1d(400, 3100)

r_cp_1=p1.line(Tcp, ch0, legend='c_{h,lf}', color="green")
r_cp_2=p1.line(Tcp, cc0, legend='c_{c,lf}', color="blue")
r_cp_3=p1.line(Tcp, ch1, legend='c_{h,hf}', color="red")
r_cp_4=p1.line(Tcp, cc1, legend='c_{c,hf}', color="purple")


p1.legend.location = "top_left"


################### Adiabatic Temperature Plot

# make MCE fun
def MCE_fun(Tdt,T,s_low,s_high,N):
    Svec=np.linspace(s_low[0]+5,s_low[-1]-5,N);
    
    T1=interp1d(s_low,T,'linear')   
    T2=interp1d(s_high,T,'linear')     
    
    DTmag_T1=T2(Svec)-T1(Svec)
    
    DT = interp1d(T1(Svec),DTmag_T1,'linear')
    return DT(Tdt)           

# Pull delta T numbers

N=75 # this increases the computational cost

Tdt=np.linspace(280,310,50)

DTHH=MCE_fun(Tdt,T,sh0,sh1,N);
DTCC=MCE_fun(Tdt,T,sc0,sc1,N);
DTHC=MCE_fun(Tdt,T,sh0,sc1,N);
DTCH=MCE_fun(Tdt,T,sc0,sh1,N);


p3 = figure(title="Adiabatic Temperature Change")
p3.xaxis.axis_label = 'Temperature (K)'
p3.yaxis.axis_label = 'Adiabtic Temperature Change (J/kgK)'
p3.grid.grid_line_alpha=0.3
p3.x_range = Range1d(260, 325)
p3.y_range = Range1d(0, 9)
r_td_1=p3.line(Tcp, DTHH[0:-1], legend='\u0394 T_{ad,HH}', color="green")
r_td_2=p3.line(Tcp, DTCC[0:-1], legend='\u0394 T_{ad,CC}', color="blue")
r_td_3=p3.line(Tcp, DTHC[0:-1], legend='\u0394 T_{ad,HC}', color="red")
r_td_4=p3.line(Tcp, DTCH[0:-1], legend='\u0394 T_{ad,CH}', color="purple")

p3.legend.location = "top_left"



# Set up the slides

def update(Thyst,k1=k0T,k2=k1T,Tshift=Tshift,Moff=Moff):
    Tc0=295
    T=np.linspace(270,330,50)
    L=45
    a=0.5
    sh0 = entropy(T,a,L,k1,Tc0+Thyst)
    sc0 = entropy(T,a,L,k1,Tc0-Thyst) 
    sh1 = entropy(T,a+Moff,L,k2,Tc0+Thyst+Tshift) 
    sc1 = entropy(T,a+Moff,L,k2,Tc0-Thyst+Tshift) 
    r1.data_source.data['y'] = sh0
    r2.data_source.data['y'] = sc0
    r3.data_source.data['y'] = sh1
    r4.data_source.data['y'] = sc1
    Tcp=T[0:-1]
    ch0= differ(T,sh0)*Tcp
    cc0= differ(T,sc0)*Tcp
    ch1= differ(T,sh1)*Tcp
    cc1= differ(T,sc1)*Tcp
    r_cp_1.data_source.data['y'] = ch0
    r_cp_2.data_source.data['y'] = cc0
    r_cp_3.data_source.data['y'] = ch1
    r_cp_4.data_source.data['y'] = cc1    
    Tdt=np.linspace(280,310,50)
    N=75
    DTHH=MCE_fun(Tdt,T,sh0,sh1,N);
    DTCC=MCE_fun(Tdt,T,sc0,sc1,N);
    DTHC=MCE_fun(Tdt,T,sh0,sc1,N);
    DTCH=MCE_fun(Tdt,T,sc0,sh1,N);
    r_td_1.data_source.data['y'] = DTHH[0:-1]
    r_td_2.data_source.data['y'] = DTCC[0:-1]
    r_td_3.data_source.data['y'] = DTHC[0:-1]
    r_td_4.data_source.data['y'] = DTCH[0:-1]   
    push_notebook()

# output notebook
output_notebook()
# show the results
show(gridplot([[p1,p2,p3]], plot_width=300, plot_height=300))  # open a browser
# interact
interact(update,Thyst=(0,.75,0.05),k0T=(-.8,-0.4),k1T=(-.8,-0.4),Tshift=(2,8),Moff=(-3,-1))

Loading BokehJS ...

interactive(children=(FloatSlider(value=0.35000000000000003, description='Thyst', max=0.75, step=0.05), FloatS…

<function __main__.update(Thyst, k1=-0.5, k2=-0.5, Tshift=6, Moff=-2)>